In [16]:
import pandas as pd
import numpy as np  

In [17]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [18]:
df=pd.read_csv("/kaggle/input/piramal/dataset/train_main_loan.csv")

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69958 entries, 0 to 69957
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                69958 non-null  object 
 1   ACCOUNT_TYPE                      69958 non-null  object 
 2   HIGH_CREDIT_OR_SANCTIONED_AMOUNT  69958 non-null  int64  
 3   DATE_OPENED                       69958 non-null  object 
 4   CURRENT_BALANCE                   69958 non-null  int64  
 5   ACTUAL_PAYMT_AMT                  65741 non-null  float64
 6   EMI_AMOUNT                        20192 non-null  float64
 7   REPAYMENT_TENURE                  32782 non-null  float64
 8   LOAN_CLASSIFICATION               69958 non-null  int64  
 9   AMOUNT_OVERDUE                    9869 non-null   float64
 10  PAYMENT_HISTORY_1                 69958 non-null  object 
 11  PAYMENT_HISTORY_2                 30285 non-null  object 
 12  OWNE

In [20]:
mode_value = df['OCCUPATION_TYPE'].mode()[0]
df['OCCUPATION_TYPE'].fillna(value=mode_value, inplace=True)
df.loc[df['OCCUPATION_TYPE'] == "R18", 'OCCUPATION_TYPE'] = "SALARIED"
unique_values = df['OCCUPATION_TYPE'].unique()
unique_values

/tmp/ipykernel_34/486089511.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['OCCUPATION_TYPE'].fillna(value=mode_value, inplace=True)


array(['SALARIED', 'SENP', 'OTHERS', 'SEP'], dtype=object)

In [21]:
df = pd.get_dummies(df, columns=['GENDER', 'OWNERSHIP_TYPE',"ACCOUNT_TYPE","OCCUPATION_TYPE"])

In [22]:
df= df.astype(int, errors='ignore')

In [23]:
df['DATE_OF_BIRTH'] = pd.to_datetime(df['DATE_OF_BIRTH'], format='%d-%m-%Y')
df['DATE_OPENED'] = pd.to_datetime(df['DATE_OPENED'], format='%d-%m-%Y')
df['age of person'] = (df['DATE_OPENED'] - df['DATE_OF_BIRTH']).dt.days / 365.25

In [24]:
df['AMOUNT_OVERDUE'].fillna(0, inplace=True)

/tmp/ipykernel_34/1861289169.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['AMOUNT_OVERDUE'].fillna(0, inplace=True)


In [25]:
df["total amount"]=df["HIGH_CREDIT_OR_SANCTIONED_AMOUNT"]-df["CURRENT_BALANCE"]+df["AMOUNT_OVERDUE"]

In [26]:
df['Reported_Date'] = pd.to_datetime(df['Reported_Date'], format='%d-%m-%Y')
df["date_opened_reported_date"]=(df['DATE_OPENED'] - df['Reported_Date']).dt.days / 365.25

In [27]:
df = df.drop(['DATE_OPENED','PAYMENT_HISTORY_START_DATE','Reported_Date','DATE_OF_BIRTH',"DATE_OF_LAST_PAYMENT","DATE_REPORTED_AND_CERTIFIED","PAYMENT_HISTORY_END_DATE"], axis=1)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69958 entries, 0 to 69957
Data columns (total 29 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                69958 non-null  object 
 1   HIGH_CREDIT_OR_SANCTIONED_AMOUNT  69958 non-null  int64  
 2   CURRENT_BALANCE                   69958 non-null  int64  
 3   ACTUAL_PAYMT_AMT                  65741 non-null  float64
 4   EMI_AMOUNT                        20192 non-null  float64
 5   REPAYMENT_TENURE                  32782 non-null  float64
 6   LOAN_CLASSIFICATION               69958 non-null  int64  
 7   AMOUNT_OVERDUE                    69958 non-null  float64
 8   PAYMENT_HISTORY_1                 69958 non-null  object 
 9   PAYMENT_HISTORY_2                 30285 non-null  object 
 10  COLLATERALVALUE                   46990 non-null  float64
 11  TU_SCORE                          69958 non-null  int64  
 12  ACTU

In [29]:
# df.drop(columns=['REPAYMENT_TENURE'], inplace=True)

In [30]:
history1=pd.read_csv("/kaggle/input/dff-new/PAYMENT_HISTORY_1.csv")
history2=pd.read_csv("/kaggle/input/dff-new/PAYMENT_HISTORY_2.csv")

In [31]:
def process_payment_history(entry):
    # Check if the entry is not a string (including NaN/None which are float in pandas)
    if not isinstance(entry, str):
        return float('nan'), 0, 0, 0, 0, 0, 0
    # Replacement rules
    replacements = {'STD': '000', 'SMA': '060', 'SUB': '090', 'DBT': '090', 'LSS': '090', 'XXX': 'XXX'}
    
    # Replace according to rules
    for key, value in replacements.items():
        entry = entry.replace(key, value)
    # Initialize counts and total for average calculation
    count_xxx = entry.count('XXX')
    nums = []  
    i = 0
    zero=0
    zero_30=0
    thirty_60=0
    sixty_90=0
    ninety_plus=0
    while i < len(entry):
        group = entry[i:i+3]  

        if 'X' in group:
            number_before_x = ''.join([char for char in group if char.isdigit()])
            if number_before_x:  
                nums.append(int(number_before_x))

            i += group.find('X')  
            while i < len(entry) and entry[i] == 'X':
                i += 1  # Skip the 'X's
        else:
            try:
                if group!="060" and group!="090":
                    nums.append(int(group))
            except ValueError:
                # This catches cases where conversion to integer might fail
                pass
            i += 3  # Move to the next group
    
    for i in range(0,len(nums)):
        num=nums[i]
        if num == 0:
            zero += 1
        elif 0 < num <= 30:
            zero_30 += 1
        elif 30 < num <= 60:
            # Check if the entry is not a string (including NaN/None which are float in pandas)
            thirty_60 += 1
        elif 60 < num <= 90:
            sixty_90 += 1
        elif num > 90:
            ninety_plus += 1
    avg_nums = sum(nums)
    if len(nums)==0:
        t=0
    else:
        t= nums[len(nums)-1]
    if len(nums)==0:
        t1=0
    else:
        t1= nums[0]
    return avg_nums, count_xxx,zero, zero_30 , thirty_60 , sixty_90,ninety_plus, t,t1


In [32]:
# Create a mask for rows in df where PAYMENT_HISTORY_1 contains 'E'
mask = df['PAYMENT_HISTORY_1'].str.contains('E')

# Create a mapping series from history_1
mapping_series = history1.set_index('ID')['PAYMENT_HISTORY_1']

# Replace values in df where mask is True by mapping using the 'ID' column
df.loc[mask, 'PAYMENT_HISTORY_1'] = df.loc[mask, 'ID'].map(mapping_series)


In [33]:
# Create a mask for rows in df where PAYMENT_HISTORY_1 contains 'E'
mask = df['PAYMENT_HISTORY_2'].notnull() & df['PAYMENT_HISTORY_2'].str.contains('E')

# Create a mapping series from history_1
mapping_series = history2.set_index('ID')['PAYMENT_HISTORY_2']

# Replace values in df where mask is True by mapping using the 'ID' column
df.loc[mask, 'PAYMENT_HISTORY_2'] = df.loc[mask, 'ID'].map(mapping_series)


In [34]:
def convert_sci_notation(s):
    try:
        # Convert to float, then to integer, and finally to string to exclude the decimal part
        return '{:.0f}'.format(float(s))
    except ValueError:
        # Return the original string if it cannot be converted to float
        return s
df['PAYMENT_HISTORY_1'] = df['PAYMENT_HISTORY_1'].apply(convert_sci_notation)
df['PAYMENT_HISTORY_2'] = df['PAYMENT_HISTORY_2'].apply(convert_sci_notation)

In [35]:
# Apply the function to the PAYMENT_HISTORY_1 column
results1 = df['PAYMENT_HISTORY_1'].apply(process_payment_history)

# Split the results into separate columns
df['avg_nums1'], df['count_xxx1'], df['zero1'], df['zero_301'], df['thirty_601'], df['sixty_901'], df['ninety_plus1'], df['firstlaste1'] ,df['lastlate1']= zip(*results1)

# Apply the function to the PAYMENT_HISTORY_2 column
results2 = df['PAYMENT_HISTORY_2'].apply(process_payment_history)

# Split the results into separate columns
df['avg_nums2'], df['count_xxx2'], df['zero2'], df['zero_302'], df['thirty_602'], df['sixty_902'], df['ninety_plus2'], df['firstlaste2'] ,df['lastlate2'] = zip(*results2)

# Calculate combined averages and counts
df['avg_nums_combined'] = (df['avg_nums1'] + df['avg_nums2']) / 36
df['count_xxx_combined'] = df['count_xxx1'] + df['count_xxx2']
df['zero_combined'] = df['zero1'] + df['zero2']
df['zero_30_combined'] = df['zero_301'] + df['zero_302']
df['thirty_60_combined'] = df['thirty_601'] + df['thirty_602']
df['sixty_90_combined'] = df['sixty_901'] + df['sixty_902']
df['ninety_plus_combined'] = df['ninety_plus1'] + df['ninety_plus2']
df['firstlaste1 combined']=df['firstlaste2']
df['lastlate1']=df['lastlate1']
# Drop the intermediate columns
df = df.drop(columns=['avg_nums1', 'count_xxx1', 'zero1', 'zero_301', 'thirty_601', 'sixty_901', 'ninety_plus1',
                      'avg_nums2', 'count_xxx2', 'zero2', 'zero_302', 'thirty_602', 'sixty_902', 'ninety_plus2',
                      'PAYMENT_HISTORY_1', 'PAYMENT_HISTORY_2','firstlaste1','firstlaste2','lastlate1','lastlate2'])


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69958 entries, 0 to 69957
Data columns (total 35 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                69958 non-null  object 
 1   HIGH_CREDIT_OR_SANCTIONED_AMOUNT  69958 non-null  int64  
 2   CURRENT_BALANCE                   69958 non-null  int64  
 3   ACTUAL_PAYMT_AMT                  65741 non-null  float64
 4   EMI_AMOUNT                        20192 non-null  float64
 5   REPAYMENT_TENURE                  32782 non-null  float64
 6   LOAN_CLASSIFICATION               69958 non-null  int64  
 7   AMOUNT_OVERDUE                    69958 non-null  float64
 8   COLLATERALVALUE                   46990 non-null  float64
 9   TU_SCORE                          69958 non-null  int64  
 10  ACTUAL_ROI                        69958 non-null  float64
 11  GENDER_Female                     69958 non-null  int64  
 12  GEND

In [37]:
df1=df.drop(columns=['ACTUAL_ROI'])

In [38]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()



# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Select only the numeric columns for normalization
numeric_cols = df1.select_dtypes(include=['int64', 'float64']).columns

# Apply the scaler to the DataFrame
df1[numeric_cols] = scaler.fit_transform(df1[numeric_cols])

In [39]:
df1[:5]

,ID,HIGH_CREDIT_OR_SANCTIONED_AMOUNT,CURRENT_BALANCE,ACTUAL_PAYMT_AMT,EMI_AMOUNT,REPAYMENT_TENURE,LOAN_CLASSIFICATION,AMOUNT_OVERDUE,COLLATERALVALUE,TU_SCORE,GENDER_Female,GENDER_Male,GENDER_Other,OWNERSHIP_TYPE_Individual,OWNERSHIP_TYPE_Joint,ACCOUNT_TYPE_Business Loan,ACCOUNT_TYPE_Housing Loan,ACCOUNT_TYPE_Personal Loan,ACCOUNT_TYPE_Property Loan,OCCUPATION_TYPE_OTHERS,OCCUPATION_TYPE_SALARIED,OCCUPATION_TYPE_SENP,OCCUPATION_TYPE_SEP,age of person,total amount,date_opened_reported_date,avg_nums_combined,count_xxx_combined,zero_combined,zero_30_combined,thirty_60_combined,sixty_90_combined,ninety_plus_combined,firstlaste1 combined
0,A002338349,0.001508,0.001236,0.000640,0.002328,0.181818,0.000000,0.00000,0.002873,0.465409,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.340078,0.012418,0.560234,0.028272,0.035714,0.529412,0.043478,0.166667,0.000000,0.057143,0.000000
1,A002000537,0.002295,0.001832,0.000037,0.003296,0.201102,0.000000,0.00000,0.002115,0.789308,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.136284,0.012764,0.489632,0.000000,0.035714,0.529412,0.000000,0.000000,0.000000,0.000000,0.000000
2,A002421579,0.003374,0.002504,0.001189,0.004330,0.190083,0.000000,0.00000,0.004941,0.676101,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.171062,0.013463,0.688335,0.000000,0.035714,0.529412,0.000000,0.000000,0.000000,0.000000,0.000000
3,A002152345,0.003414,0.003329,0.002002,NaN,NaN,0.000000,0.00000,0.004200,0.795597,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.480826,0.012258,0.956621,0.000000,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000
4,A001952834,0.004285,0.003458,0.001864,NaN,0.146006,0.598889,0.00139,0.003836,0.411950,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.298196,0.014737,0.576771,0.622458,0.035714,0.000000,0.000000,0.055556,0.058824,0.942857,0.596774


In [59]:
# import numpy as np
# from sklearn.impute import KNNImputer
# imputer = KNNImputer(n_neighbors=2, weights="uniform")

# # Applying imputer on selected features
# # Assuming 'df' is your original DataFrame and it includes all the necessary columns
# selected_columns = [ 'ACTUAL_PAYMT_AMT', 'EMI_AMOUNT', 'COLLATERALVALUE','REPAYMENT_TENURE','HIGH_CREDIT_OR_SANCTIONED_AMOUNT','CURRENT_BALANCE','OWNERSHIP_TYPE_Individual','OWNERSHIP_TYPE_Joint','ACCOUNT_TYPE_Personal Loan']

# imputed_data = imputer.fit_transform(df[selected_columns])

# # Convert the imputed data back to a DataFrame (optional step to replace only the selected columns in original DataFrame)
# imputed_df = pd.DataFrame(imputed_data, columns=selected_columns)

# # If you want to replace the original columns in df with the imputed values
# df[selected_columns] = imputed_df

In [40]:
df_miss = df.drop(columns=['ACTUAL_ROI'])

In [41]:
df1.isnull().sum()

ID                                      0
HIGH_CREDIT_OR_SANCTIONED_AMOUNT        0
CURRENT_BALANCE                         0
ACTUAL_PAYMT_AMT                     4217
EMI_AMOUNT                          49766
REPAYMENT_TENURE                    37176
LOAN_CLASSIFICATION                     0
AMOUNT_OVERDUE                          0
COLLATERALVALUE                     22968
TU_SCORE                                0
GENDER_Female                           0
GENDER_Male                             0
GENDER_Other                            0
OWNERSHIP_TYPE_Individual               0
OWNERSHIP_TYPE_Joint                    0
ACCOUNT_TYPE_Business Loan              0
ACCOUNT_TYPE_Housing Loan               0
ACCOUNT_TYPE_Personal Loan              0
ACCOUNT_TYPE_Property Loan              0
OCCUPATION_TYPE_OTHERS                  0
OCCUPATION_TYPE_SALARIED                0
OCCUPATION_TYPE_SENP                    0
OCCUPATION_TYPE_SEP                     0
age of person                     

In [42]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [43]:
imputer = IterativeImputer(max_iter=10, random_state=0)

In [44]:
selected_columns = [ 'ACTUAL_PAYMT_AMT', 'EMI_AMOUNT', 'COLLATERALVALUE','REPAYMENT_TENURE','HIGH_CREDIT_OR_SANCTIONED_AMOUNT','CURRENT_BALANCE','OWNERSHIP_TYPE_Individual','OWNERSHIP_TYPE_Joint','ACCOUNT_TYPE_Personal Loan']

In [45]:
imputed_data = imputer.fit_transform(df1[selected_columns])

/opt/conda/lib/python3.10/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [46]:
imputed_df = pd.DataFrame(imputed_data, columns=selected_columns)

In [47]:
df1[selected_columns] = imputed_df

In [48]:
df1.isnull().sum()

ID                                  0
HIGH_CREDIT_OR_SANCTIONED_AMOUNT    0
CURRENT_BALANCE                     0
ACTUAL_PAYMT_AMT                    0
EMI_AMOUNT                          0
REPAYMENT_TENURE                    0
LOAN_CLASSIFICATION                 0
AMOUNT_OVERDUE                      0
COLLATERALVALUE                     0
TU_SCORE                            0
GENDER_Female                       0
GENDER_Male                         0
GENDER_Other                        0
OWNERSHIP_TYPE_Individual           0
OWNERSHIP_TYPE_Joint                0
ACCOUNT_TYPE_Business Loan          0
ACCOUNT_TYPE_Housing Loan           0
ACCOUNT_TYPE_Personal Loan          0
ACCOUNT_TYPE_Property Loan          0
OCCUPATION_TYPE_OTHERS              0
OCCUPATION_TYPE_SALARIED            0
OCCUPATION_TYPE_SENP                0
OCCUPATION_TYPE_SEP                 0
age of person                       0
total amount                        0
date_opened_reported_date           0
avg_nums_com

In [49]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69958 entries, 0 to 69957
Data columns (total 34 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                69958 non-null  object 
 1   HIGH_CREDIT_OR_SANCTIONED_AMOUNT  69958 non-null  float64
 2   CURRENT_BALANCE                   69958 non-null  float64
 3   ACTUAL_PAYMT_AMT                  69958 non-null  float64
 4   EMI_AMOUNT                        69958 non-null  float64
 5   REPAYMENT_TENURE                  69958 non-null  float64
 6   LOAN_CLASSIFICATION               69958 non-null  float64
 7   AMOUNT_OVERDUE                    69958 non-null  float64
 8   COLLATERALVALUE                   69958 non-null  float64
 9   TU_SCORE                          69958 non-null  float64
 10  GENDER_Female                     69958 non-null  float64
 11  GENDER_Male                       69958 non-null  float64
 12  GEND

In [50]:
# spearman_corr = df.corr(method='spearman')
# kendall_tau = df.corr(method='kendall')

In [51]:
# X=kendall_tau["EMI_AMOUNT"]
# X=X[abs(X)>0.4]
# X

In [52]:
# X=spearman_corr["EMI_AMOUNT"]
# X=X[abs(X)>0.4]
# X

In [53]:
# # Calculate correlation with 'ACTUAL_ROI'
# # Calculate correlation with 'ACTUAL_ROI'
# corr_with_roi = df.corr()['ACTUAL_ROI'].sort_values(ascending=False) = df.corr()['ACTUAL_ROI'].sort_values(ascending=False)

In [54]:
# spearman_corr = df.corr(method='spearman')
# kendall_tau = df.corr(method='kendall')

In [55]:
# spearman_corr["ACTUAL_ROI"]

In [56]:
# # corr_with_roi = df.corr()['ACTUAL_ROI'].sort_values(ascending=False) = df.corr()['ACTUAL_ROI'].sort_values(ascending=False)
# corr_with_roi

In [57]:
df1.drop(columns=['ID'], inplace=True)

In [35]:
# df = df[df['ACTUAL_ROI'] != 0]


In [58]:
y=df["ACTUAL_ROI"]



In [59]:
y=y.to_numpy()
X=df1.to_numpy()

In [60]:
print(X.shape)
print(y.shape)

(69958, 33)
(69958,)


In [61]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [62]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np

param_distributions = {
    'n_estimators': [100, 200, 500, 1000],
    'max_depth': [6, 10, 15, 20],
    'min_child_weight': [1, 2, 5, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.5, 1, 1.5, 2],
    'reg_alpha': [0, 0.1, 0.5, 1, 2],
    'reg_lambda': [0.1, 1, 5, 10],
    'scale_pos_weight': [1, 2, 5],
    'max_delta_step': [0, 1, 2, 5],
}
# Initialize the XGBRegressor model with GPU support
# xgb = XGBRegressor(tree_method='gpu_hist', gpu_id=0)  # Specify tree_method as 'gpu_hist' to use GPU
xgb = XGBRegressor(tree_method='hist', device='cuda')

# Setup RandomizedSearchCV
xgb_random = RandomizedSearchCV(estimator=xgb, param_distributions=param_distributions, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model
xgb_random.fit(X_train, y_train)

# Best parameters
print("Best parameters found: ", xgb_random.best_params_)

# Predict on the validation data using the best parameters
y_pred = xgb_random.predict(X_val)

# Calculate the RMSE for the validation data
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred))

print("Validation RMSE: ", rmse_val)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [15:35:57] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [15:35:57] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for boo

[CV] END colsample_bytree=0.9, gamma=0, learning_rate=0.05, max_delta_step=1, max_depth=6, min_child_weight=5, n_estimators=1000, reg_alpha=0.5, reg_lambda=0.1, scale_pos_weight=5, subsample=1.0; total time=  18.1s
[CV] END colsample_bytree=1.0, gamma=1, learning_rate=0.2, max_delta_step=1, max_depth=6, min_child_weight=2, n_estimators=100, reg_alpha=0.5, reg_lambda=5, scale_pos_weight=1, subsample=0.7; total time=   1.8s
[CV] END colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=6, min_child_weight=10, n_estimators=100, reg_alpha=0.5, reg_lambda=10, scale_pos_weight=2, subsample=0.8; total time=   1.8s
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=10, min_child_weight=5, n_estimators=100, reg_alpha=0.1, reg_lambda=1, scale_pos_weight=5, subsample=1.0; total time=   4.5s
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=10, min_child_weight=5, n_estimators=100, reg_alpha=0.1, re

/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [15:45:01] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=0.8, gamma=2, learning_rate=0.2, max_delta_step=2, max_depth=15, min_child_weight=1, n_estimators=1000, reg_alpha=0.1, reg_lambda=1, scale_pos_weight=5, subsample=0.9; total time=  15.1s
[CV] END colsample_bytree=1.0, gamma=1, learning_rate=0.2, max_delta_step=1, max_depth=6, min_child_weight=2, n_estimators=100, reg_alpha=0.5, reg_lambda=5, scale_pos_weight=1, subsample=0.7; total time=   1.8s
[CV] END colsample_bytree=1.0, gamma=1, learning_rate=0.2, max_delta_step=1, max_depth=6, min_child_weight=2, n_estimators=100, reg_alpha=0.5, reg_lambda=5, scale_pos_weight=1, subsample=0.7; total time=   1.7s
[CV] END colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=6, min_child_weight=10, n_estimators=100, reg_alpha=0.5, reg_lambda=10, scale_pos_weight=2, subsample=0.8; total time=   1.8s
[CV] END colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=6, min_child_weight=10, n_estimators=100, reg_alpha=0.5, reg_la

...

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [15:55:13] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=0.8, gamma=0.2, learning_rate=0.05, max_delta_step=0, max_depth=10, min_child_weight=1, n_estimators=1000, reg_alpha=1, reg_lambda=1, scale_pos_weight=1, subsample=0.6; total time=  42.3s
[CV] END colsample_bytree=1.0, gamma=1.5, learning_rate=0.01, max_delta_step=1, max_depth=10, min_child_weight=1, n_estimators=200, reg_alpha=0.5, reg_lambda=0.1, scale_pos_weight=1, subsample=0.9; total time=   4.7s
[CV] END colsample_bytree=0.8, gamma=2, learning_rate=0.05, max_delta_step=1, max_depth=15, min_child_weight=1, n_estimators=100, reg_alpha=2, reg_lambda=1, scale_pos_weight=1, subsample=1.0; total time=   5.7s
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6, min_child_weight=2, n_estimators=100, reg_alpha=1, reg_lambda=5, scale_pos_weight=1, subsample=0.9; total time=   2.1s
[CV] END colsample_bytree=0.7, gamma=1.5, learning_rate=0.1, max_delta_step=5, max_depth=15, min_child_weight=5, n_estimators=200, reg_alpha=2, reg_l

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [15:58:35] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [15:59:58] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for boo

Best parameters found:  {'subsample': 1.0, 'scale_pos_weight': 2, 'reg_lambda': 10, 'reg_alpha': 0, 'n_estimators': 500, 'min_child_weight': 5, 'max_depth': 10, 'max_delta_step': 0, 'learning_rate': 0.05, 'gamma': 0, 'colsample_bytree': 0.6}
Validation RMSE:  2.112677842991658


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:00:18] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [63]:

# Predict on the validation data using the best parameters
y_pred = xgb_random.predict(X_val)

# Calculate the RMSE for the validation data
rmse_train = np.sqrt(mean_squared_error(y_val, y_pred))

print("validation RMSE: ", rmse_train)

validation RMSE:  2.112677842991658


In [64]:
import numpy as np
y_pred = xgb_random.predict(X_val)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_mask = y_true != 0
    return np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100
mm=y_val==0
print(mm.sum())
# Assuming y_train are the true values and y_pred are the predicted values from your model
mape_train = mean_absolute_percentage_error(y_val, y_pred)

print("validation MAPE: {:.2f}%".format(mape_train))


0
validation MAPE: 7.25%


In [65]:


from joblib import dump

# Assuming xgb_random is your trained RandomizedSearchCV model
# Save the model to a file
dump(xgb_random, 'xgb_7.10MAPE.joblib')

['xgb_7.10MAPE.joblib']

[CV] END colsample_bytree=1.0, gamma=2, learning_rate=0.01, max_delta_step=2, max_depth=20, min_child_weight=2, n_estimators=1000, reg_alpha=0, reg_lambda=10, scale_pos_weight=1, subsample=0.7; total time= 1.7min
[CV] END colsample_bytree=0.6, gamma=1, learning_rate=0.1, max_delta_step=2, max_depth=6, min_child_weight=10, n_estimators=1000, reg_alpha=0.5, reg_lambda=10, scale_pos_weight=2, subsample=0.8; total time=  17.5s
[CV] END colsample_bytree=0.9, gamma=0, learning_rate=0.05, max_delta_step=1, max_depth=10, min_child_weight=10, n_estimators=200, reg_alpha=2, reg_lambda=1, scale_pos_weight=5, subsample=0.9; total time=   9.7s
[CV] END colsample_bytree=0.9, gamma=0, learning_rate=0.05, max_delta_step=1, max_depth=10, min_child_weight=10, n_estimators=200, reg_alpha=2, reg_lambda=1, scale_pos_weight=5, subsample=0.9; total time=  10.7s
[CV] END colsample_bytree=0.9, gamma=0, learning_rate=0.05, max_delta_step=1, max_depth=10, min_child_weight=10, n_estimators=200, reg_alpha=2, reg_l

In [ ]:
newdf.isna().sum()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from keras.models import Sequential
from keras.layers import Dense
from xgboost import XGBClassifier
import numpy as np

In [ ]:
from sklearn.ensemble import BaggingRegressor

def create_subsets(X, y, n_subsets):
    # Use BaggingRegressor for regression tasks
    bagging = BaggingRegressor(n_estimators=n_subsets, max_samples=1.0/n_subsets, random_state=42)
    bagging.fit(X, y)
    
    # Generate subsets based on the samples chosen by the bagging regressor
# Adjusted for numpy arrays
    subsets = [(X[subset_indices], y[subset_indices]) for subset_indices in bagging.estimators_samples_]
    
    return subsets


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

def train_dnn(subset_x, subset_y, num_features=29, num_neurons=[64, 32], dropout_rate=0.2, learning_rate=0.001):
    # Define the DNN structure for regression
    model = Sequential()
    
    # Input layer
    model.add(Dense(num_neurons[0], input_dim=num_features, activation='relu'))
    model.add(Dropout(dropout_rate))
    
    # Hidden layers
    for neurons in num_neurons[1:]:
        model.add(Dense(neurons, activation='relu'))
        model.add(Dropout(dropout_rate))
    
    # Output layer for a regression task - no activation function or 'linear' could be used
    model.add(Dense(1, activation='linear'))
    
    # Compile the model for a regression task
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    
    # Train the model on the subset
    model.fit(subset_x, subset_y, epochs=50, batch_size=32, verbose=0)
    
    # Return the model including the weights of the last hidden layer
    # This model can be used as a feature extractor
    feature_extractor = Sequential()
    for layer in model.layers[:-1]:  # exclude the output layer
        feature_extractor.add(layer)
    
    return feature_extractor


In [ ]:
def train_xgboost(features, labels):
    # Initialize XGBRegressor with GPU support
    model = XGBRegressor(objective='reg:squarederror', tree_method='gpu_hist')
    model.fit(features, labels)
    return model


In [ ]:
import numpy as np

def ensemble_predictions(models, test_features):
    # Aggregate predictions from each model by averaging
    predictions = np.array([model.predict(test_features) for model in models])
    avg_prediction = np.mean(predictions, axis=0)
    return avg_prediction


In [ ]:
# Assume X and y are your features and labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
subsets = create_subsets(X_train, y_train, n_subsets=10)  # number of subsets
print(len(subsets[0][0]))

In [ ]:
subsets = create_subsets(X_train, y_train, n_subsets=10)  # number of subsets


In [ ]:

# Adjusted call to train_dnn within the list comprehension
dnn_features = [train_dnn(subset_x, subset_y) for subset_x, subset_y in subsets]


In [ ]:
list=[]
for _,z in subsets:
    list.append(z)

In [ ]:
single_array_y = np.concatenate(list, axis=0)

In [ ]:
single_array_y.shape

In [ ]:
for i in range(0,10):
    z=dnn_features[i]
    dnn_features[i]=z.predict(subsets[i][0])

In [ ]:
for i in range(0,len(dnn_features)):
    print(dnn_features[i].shape)

In [ ]:
single_array = np.concatenate(dnn_features, axis=0)

In [ ]:
single_array.shape

In [ ]:
xgboost_models = [train_xgboost(features, y_train) for features in dnn_features]


In [ ]:

# Get the ensemble prediction
final_prediction = ensemble_predictions(xgboost_models, single_array)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# Assuming y_test are the true values and y_pred are the predicted values from your ensemble model
# y_pred = ensemble_predictions(xgboost_models, X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(single_array, final_prediction))
print(f"Root Mean Squared Error (RMSE): {rmse}")

# # Define MAPE function
# def mape(y_true, y_pred):
#     y_true, y_pred = np.array(y_true), np.array(y_pred)
#     non_zero_mask = y_true != 0  # Mask to avoid division by zero
#     return np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100

# # Calculate MAPE
# mape_value = mape(y_test, y_pred)
# print(f"Mean Absolute Percentage Error (MAPE): {mape_value}%")


In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np

param_distributions = {
    'n_estimators': [100, 200, 500, 1000],
    'max_depth': [6, 10, 15, 20],
    'min_child_weight': [1, 2, 5, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.5, 1, 1.5, 2],
    'reg_alpha': [0, 0.1, 0.5, 1, 2],
    'reg_lambda': [0.1, 1, 5, 10],
    'scale_pos_weight': [1, 2, 5],
    'max_delta_step': [0, 1, 2, 5],
}
# Initialize the XGBRegressor model with GPU support
# xgb = XGBRegressor(tree_method='gpu_hist', gpu_id=0)  # Specify tree_method as 'gpu_hist' to use GPU
xgb = XGBRegressor(tree_method='hist', device='cuda')

# Setup RandomizedSearchCV
xgb_random = RandomizedSearchCV(estimator=xgb, param_distributions=param_distributions, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model
xgb_random.fit(X_train, y_train)

# Best parameters
print("Best parameters found: ", xgb_random.best_params_)

# Predict on the validation data using the best parameters
y_pred = xgb_random.predict(X_val)

# Calculate the RMSE for the validation data
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred))

print("Validation RMSE: ", rmse_val)


In [ ]:
# Initialize the XGBRegressor model with GPU support
# xgb = XGBRegressor(tree_method='gpu_hist', gpu_id=0)  # Specify tree_method as 'gpu_hist' to use GPU
xgb = XGBRegressor(tree_method='hist', device='cuda')

# Setup RandomizedSearchCV
xgb_random = RandomizedSearchCV(estimator=xgb, param_distributions=param_distributions, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Fit the random search model
xgb_random.fit(X_train, y_train)

# Best parameters
print("Best parameters found: ", xgb_random.best_params_)

# Predict on the validation data using the best parameters
y_pred = xgb_random.predict(X_val)

# Calculate the RMSE for the validation data
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred))

print("Validation RMSE: ", rmse_val)


In [ ]:

# Predict on the validation data using the best parameters
y_pred = xgb_random.predict(X_train)

# Calculate the RMSE for the validation data
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred))

print("training RMSE: ", rmse_train)
import numpy as np
y_pred = xgb_random.predict(X_val)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_mask = y_true != 0
    return np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100
mm=y_val==0
print(mm.sum())
# Assuming y_train are the true values and y_pred are the predicted values from your model
mape_train = mean_absolute_percentage_error(y_val, y_pred)

print("validation MAPE: {:.2f}%".format(mape_train))


In [ ]:
import numpy as np
y_pred = xgb_random.predict(X_val)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_mask = y_true != 0
    return np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100
mm=y_val==0
print(mm.sum())
# Assuming y_train are the true values and y_pred are the predicted values from your model
mape_train = mean_absolute_percentage_error(y_val, y_pred)

print("validation MAPE: {:.2f}%".format(mape_train))


In [ ]:
x = np.where(
    df['CURRENT_BALANCE'].notnull() & df['ACTUAL_PAYMT_AMT'].notnull(),
    df['CURRENT_BALANCE'] / df['ACTUAL_PAYMT_AMT'],
    np.nan
)

In [ ]:
arr=x
arr = arr[~np.isnan(arr)]

percentiles = [np.percentile(arr, p) for p in range(10, 100, 10)]
print(percentiles)

In [ ]:
numbers_greater_than_percentiles = []

for percentile in range(10, 100, 10):
    percentile_value = np.nanpercentile(arr, percentile)
    # Choose a number just greater than the calculated percentile
    numbers_greater_than_percentiles.append(percentile_value + 0.01)  # Adding a small delta

numbers_greater_than_percentiles

In [ ]:
import random
df['ACTUAL_PAYMT_AMT'].fillna(df['CURRENT_BALANCE'] / random.randint(72, 91), inplace=True)

In [ ]:
x = np.where(
    df['CURRENT_BALANCE'].notnull() & df['COLLATERALVALUE'].notnull(),
    df['COLLATERALVALUE']/df['CURRENT_BALANCE'],
    np.nan
)

In [ ]:
arr=x
arr = arr[~np.isnan(arr)]

percentiles = [np.percentile(arr, p) for p in range(10, 100, 10)]
print(percentiles)

In [ ]:
numbers_greater_than_percentiles = []

for percentile in range(10, 100, 10):
    percentile_value = np.nanpercentile(arr, percentile)
    # Choose a number just greater than the calculated percentile
    numbers_greater_than_percentiles.append(percentile_value + 0.01)  # Adding a small delta

numbers_greater_than_percentiles

In [ ]:
df['COLLATERALVALUE'].fillna(df['CURRENT_BALANCE'] * random.uniform(1.8, 2.5), inplace=True)
df['REPAYMENT_TENURE'].fillna(df['HIGH_CREDIT_OR_SANCTIONED_AMOUNT'] / random.randint(6600, 12000), inplace=True)

In [ ]:
x = np.where(
    df['REPAYMENT_TENURE'].notnull() & df['HIGH_CREDIT_OR_SANCTIONED_AMOUNT'].notnull(),
    df['HIGH_CREDIT_OR_SANCTIONED_AMOUNT']/df['REPAYMENT_TENURE'],
    np.nan
)

In [ ]:
arr=x
arr = arr[~np.isnan(arr)]

percentiles = [np.percentile(arr, p) for p in range(10, 100, 10)]
print(percentiles)

In [ ]:
df['REPAYMENT_TENURE'].fillna(df['HIGH_CREDIT_OR_SANCTIONED_AMOUNT'] / random.randint(6600, 12000), inplace=True)

In [ ]:
mode_values = df['DATE_OF_LAST_PAYMENT'].mode().head()

# Replace null values with one of the top 5 mode values randomly
null_indices = df[df['DATE_OF_LAST_PAYMENT'].isnull()].index
for index in null_indices:
    df.at[index, 'DATE_OF_LAST_PAYMENT'] = np.random.choice(mode_values)


In [ ]:
null_counts = df.isnull().sum()
# Printing the count of null values for each column
print(null_counts)

In [ ]:
df[:5]

In [ ]:
def process_payment_history(entry):
    # Replacement rules
    replacements = {'STD': '000', 'SMA': '060', 'SUB': '090', 'DBT': '090', 'LSS': '090', 'XXX': 'XXX'}
    
    # Replace according to rules
    for key, value in replacements.items():
        entry = entry.replace(key, value)
    
    # Initialize counts and total for average calculation
    count_xxx = entry.count('XXX')
    count_060 = entry.count('060')
    count_090 = entry.count('090')
    nums = []  
    i = 0

    while i < len(entry):
        group = entry[i:i+3]  

        if 'X' in group:
            number_before_x = ''.join([char for char in group if char.isdigit()])
            if number_before_x:  
                nums.append(int(number_before_x))

            i += group.find('X')  
            while i < len(entry) and entry[i] == 'X':
                i += 1  # Skip the 'X's
        else:
            try:
                if group!="060" and group!="090":
                    nums.append(int(group))
            except ValueError:
                # This catches cases where conversion to integer might fail
                pass
            i += 3  # Move to the next group
    
    avg_nums = sum(nums) / len(nums) if nums else 0

    return avg_nums, count_xxx, count_060, count_090



In [ ]:
# Apply the function to the DataFrame
results = df['PAYMENT_HISTORY_1'].apply(process_payment_history)

# Split the results into separate columns
df['avg_nums'], df['count_xxx'], df['count_060'], df['count_090'] = zip(*results)
# Apply the function to the DataFrame
results = df['PAYMENT_HISTORY_2'].apply(process_payment_history)

# Split the results into separate columns
df['avg_nums2'], df['count_xxx2'], df['count_0602'], df['count_0902'] = zip(*results)


In [ ]:
# Apply the function to the DataFrame
results = df['PAYMENT_HISTORY_2'].apply(process_payment_history)

# Split the results into separate columns
df['avg_nums2'], df['count_xxx2'], df['count_0602'], df['count_0902'] = zip(*results)


In [ ]:
df = df.drop(['PAYMENT_HISTORY_2', 'PAYMENT_HISTORY_1'], axis=1)

In [ ]:
df[:5]

In [ ]:
df['DATE_OF_BIRTH'] = pd.to_datetime(df['DATE_OF_BIRTH'], format='%d-%m-%Y')
df['DATE_OPENED'] = pd.to_datetime(df['DATE_OPENED'], format='%d-%m-%Y')

# Calculate the age at the time of account opening
df['age_at_opening'] = df['DATE_OPENED'].dt.year - df['DATE_OF_BIRTH'].dt.year


In [ ]:
df = df.drop(['DATE_OPENED', 'DATE_OF_BIRTH'], axis=1)

In [ ]:
df.info()

In [ ]:
df = df.drop(['DATE_OF_LAST_PAYMENT'], axis=1)

In [ ]:
bool_columns = df.select_dtypes(include=bool).columns
df[bool_columns] = df[bool_columns].astype(int)

In [ ]:
df[:5]

In [ ]:
2.5


In [ ]:
newdf.info()

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import RootMeanSquaredError


In [ ]:
y=newdf["ACTUAL_ROI"]
newdf=newdf.drop("ACTUAL_ROI",axis=1)
y=y.to_numpy()
X=newdf.to_numpy()


In [ ]:
y=y.to_numpy()

In [ ]:
newdf=newdf.to_numpy()


In [ ]:
X=newdf

In [ ]:
# Assuming a regression task, change the output layer and loss function accordingly
model = Sequential([
    Dense(128, activation='relu', input_shape=(40,)),  # Assuming 40 input features
    Dense(64, activation='relu'),
    Dense(1, activation='linear')  # Output layer for regression
])

# Compile the model with RMSE as a metric
model.compile(optimizer='adam',
              loss='mean_squared_error',  # Use MSE loss for regression tasks
              metrics=[RootMeanSquaredError(name='rmse')])  # Add RMSE as a metric

# Summary of the model
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
history = model.fit(X_train, y_train,          # X_train is your input features, y_train are the labels
                    batch_size=32,             # Batch size (optional, default is 32)
                    epochs=10,                 # Number of epochs
                    validation_data=(X_val, y_val),  # Validation data (optional)
                    verbose=1)    

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming X, y are your features and target variable, respectively

# Split data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the RandomForestRegressor model
model = RandomForestRegressor()

# Fit the model on the training data
model.fit(X_train, y_train)

# Predict on the validation data
y_pred = model.predict(X_val)

# Calculate the RMSE for the validation data
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred))

print("Validation RMSE: ", rmse_val)


In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV


In [ ]:
# Hyperparameter grid
param_distributions = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt'],
    'bootstrap': [True, False]
}

# Initialize the RandomForestRegressor model
rf = RandomForestRegressor()

# Setup RandomizedSearchCV
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = param_distributions, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rf_random.fit(X_train, y_train)

# Best parameters
print("Best parameters found: ", rf_random.best_params_)

# Predict on the validation data using the best parameters
y_pred = rf_random.predict(X_val)

# Calculate the RMSE for the validation data
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred))

print("Validation RMSE: ", rmse_val)

In [ ]:
from joblib import dump, load

# Save the model to a file
dump(model, 'random_forest_regressor.joblib')

# Now, you can load it back at any time with
loaded_model = load('random_forest_regressor.joblib')

# And then use it to make predictions
# y_pred_loaded = loaded_model.predict(X_val)
